# [Title]

## Introduction

Music is a form of art that is ubiquitous and has a rich history. Different composers have created music with their unique styles and compositions. However, identifying the composer of a particular piece of music can be a challenging task, especially for novice musicians or listeners. The proposed project aims to use deep learning techniques to identify the composer of a given piece of music accurately.

The primary objective of this project is to develop a deep learning model that can predict the composer of a given musical score accurately. The project aims to accomplish this objective by using two deep learning techniques: Long Short-Term Memory (LSTM) and Convolutional Neural Network (CNN).

## Library Imports

In [5]:
import numpy as np
import pandas as pd
import os
import glob
import pretty_midi
import matplotlib.pyplot as plt
import seaborn as sns

## Data Collection

### Loading Training Data

In [8]:
train_files = glob.glob('./Composer_Dataset/NN_midi_files_extended/train/*/*.mid')
train_data = []

for train_file in train_files:
    try:
        midi_file = pretty_midi.PrettyMIDI(train_file) 
        train_data.append((os.path.basename(os.path.dirname(train_file)), midi_file)) 
    except (IOError, Exception):
        pass 



for path in train_files:
    try:
        midi = pretty_midi.PrettyMIDI(path)
        composer = os.path.basename(os.path.dirname(path))
        train_data.append((composer, midi))
    except Exception as e:
        # Optionally log the error if needed
        # print(f"Skipping {path}: {e}")
        continue

/opt/anaconda3/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [9]:
train_data

[('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x118162630>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x1295ea180>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x129c0b4d0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x12f90c8c0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x12a574b00>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x103d93950>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x12f939e20>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x12f8281d0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x139c21c40>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x139945970>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x13996c170>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x12a6fb500>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x11812e2a0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x139c97ad0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x12a4a4260>),
 ('mozart'

### Loading Test Data

In [11]:
test_files = glob.glob('./Composer_Dataset/NN_midi_files_extended/test/*/*.mid')
test_data = []

for test_file in train_files:
    try:
        midi_file = pretty_midi.PrettyMIDI(test_file) 
        test_data.append((os.path.basename(os.path.dirname(test_file)), midi_file)) 
    except (IOError, Exception):
        pass 

In [12]:
test_data

[('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x12a4495b0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x326de4170>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x326918b00>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x326dec770>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x326a6aff0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x32861c380>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x328bf32c0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x32844b5c0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x328b43770>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x328b20050>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x32884bbc0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x327c4d3a0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x326943380>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x326de4140>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x32ad28d70>),
 ('mozart'

### Loading Dev Data

In [14]:
dev_files = glob.glob('./Composer_Dataset/NN_midi_files_extended/dev/*/*.mid')
dev_data = []

for dev_file in train_files:
    try:
        midi_file = pretty_midi.PrettyMIDI(dev_file) 
        dev_data.append((os.path.basename(os.path.dirname(dev_file)), midi_file)) 
    except (IOError, Exception):
        pass 

In [15]:
dev_data

[('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x10fa224b0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x368236690>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x3697a39b0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x368220500>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x36951cb90>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x11807d340>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x369d655b0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x36a42a030>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x369f8fa10>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x369fee510>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x1181634a0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x36968f080>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x369d937a0>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x368217530>),
 ('mozart', <pretty_midi.pretty_midi.PrettyMIDI at 0x36c65eed0>),
 ('mozart'

## Data Pre-processing: 
Convert the musical scores into a format suitable for deep learning models. This involves converting the musical scores into MIDI files and applying data augmentation techniques.


## Feature Extraction: 
Extract features from the MIDI files, such as notes, chords, and tempo, using music analysis tools.


## Model Building: 
Develop a deep learning model using LSTM and CNN architectures to classify the musical scores according to the composer.


## Model Training: 
Train the deep learning model using the pre-processed and feature-extracted data.


## Model Evaluation: 
Evaluate the performance of the deep learning model using accuracy, precision, and recall metrics.


## Model Optimization: 
Optimize the deep learning model by fine-tuning hyperparameters.